# Automated Test Script

#### Importing required libraries

In [3]:
import numpy as np
import pandas as pd
import sys, os

import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

#### Locating our logs files in our working directory

In [4]:
LOGS_PATH = os.path.join(os.getcwd(), 'logs')

#### Replace this with the log file name that your looking to test

In [5]:
LOG_BARCODE = '2018-04-07_19-36'

#### Loading DeepEye & Tester log files into Pandas DataFrames

In [6]:
tester_log = pd.read_csv(os.path.join(LOGS_PATH, "[" + LOG_BARCODE + "]--DeepEye.csv"), index_col=0)
deepeye_log = pd.read_csv(os.path.join(LOGS_PATH, "[" + LOG_BARCODE + "]--Tester.csv"), index_col=0) 

## Compare the two dataframes and identify each feature where the `GROUND TRUTH` and the `PREDICTION` by DeepEye were different

In [7]:
results = pd.concat([tester_log.set_index('FRAME_ID'), deepeye_log.set_index('FRAME_ID')], 
                   axis='columns', keys=['GROUND TRUTH', 'PREDICTION'])

results = results.swaplevel(axis='columns')[tester_log.columns[1:]]
results

PEDESTRIAN                VEHICLES                   BIKES  \
         GROUND TRUTH PREDICTION GROUND TRUTH PREDICTION GROUND TRUTH   
FRAME_ID                                                                
0                   1          1            1          1            0   
1                   0          1            1          1            0   
2                   0          1            1          1            0   
3                   1          1            1          1            0   
4                   0          1            1          1            0   
5                   1          1            1          1            0   
6                   1          1            1          1            0   
7                   1          1            1          1            0   
8                   0          1            1          1            0   
9                   1          1            1          1            0   
10                  1          1            1          1            0   
11                  1          1            1          1            0   
12                  0          1            1          1            0   
13                  1          1            1          1            0   
14                  1          1            1          1            0   
15                  1          1            1          1            0   
16                  0          0            1          1            1   
17                  1          1            1          1            0   
18                  0          0            1          1            0   
19                  0          1            1          1            0   
20                  0          1            1          1            0   
21                  0          1            1          1            0   
22                  0          1            1          1            1   
23                  1          1            0          0            1   
24                  1          1            1          1            1   
25                  0          1            1          1            0   
26                  0          0            1          1            0   
27                  1          1            1          1            0   
28                  1          1            1          1            0   
29                  0          0            1          1            0   
...               ...        ...          ...        ...          ...   
244                 0          0            1          1            0   
245                 0          0            1          1            0   
246                 0          0            0          1            0   
247                 0          0            1          1            0   
248                 0          0            1          1            0   
249                 0          0            1          1            0   
250                 0          0            1          1            0   
251                 1          0            1          1            0   
252                 0          0            1          1            0   
253                 0          0            1          1            0   
254                 0          0            1          1            0   
255                 0          0            0          1            0   
256                 0          1            0          1            0   
257                 0          0            1          1            0   
258                 0          0            1          1            0   
259                 0          0            0          1            0   
260                 0          0            1          1            0   
261                 0          0            1          1            0   
262                 0          0            1          1            0   
263                 0          0            1          1            0   
264                 0          0            1          1            0

#### Look at False Positive/Negative Predictions

In [8]:
log = (tester_log != deepeye_log).stack()
errors = log[log]
errors.index.names = ['FRAME_ID', 'FEATURE']
difference_loc = np.where(tester_log != deepeye_log)
ground_truth = tester_log.values[difference_loc]
prediction = deepeye_log.values[difference_loc]
comparison_table = pd.DataFrame({'GROUND TRUTH': ground_truth, 'PREDICTION': prediction}, index=errors.index)

In [9]:
comparison_table

GROUND TRUTH  PREDICTION
FRAME_ID FEATURE                                
0        TRAFFIC_LIGHT             0           1
1        PEDESTRIAN                0           1
         TRAFFIC_LIGHT             0           1
2        PEDESTRIAN                0           1
         TRAFFIC_LIGHT             0           1
         OFF_LANE                  1           0
3        TRAFFIC_LIGHT             0           1
4        PEDESTRIAN                0           1
         TRAFFIC_LIGHT             0           1
5        TRAFFIC_LIGHT             0           1
6        TRAFFIC_LIGHT             0           1
7        TRAFFIC_LIGHT             0           1
8        PEDESTRIAN                0           1
         TRAFFIC_LIGHT             0           1
9        TRAFFIC_LIGHT             0           1
10       TRAFFIC_LIGHT             0           1
         OFF_LANE                  1           0
11       TRAFFIC_LIGHT             0           1
         OFF_LANE                  1           0
         COLLISION                 0           1
12       PEDESTRIAN                0           1
         TRAFFIC_LIGHT             0           1
         OFF_LANE                  1           0
         COLLISION                 0           1
13       TRAFFIC_LIGHT             0           1
15       BIKES                     0           1
         OFF_LANE                  0           1
16       TRAFFIC_LIGHT             0           1
17       TRAFFIC_LIGHT             0           1
18       TRAFFIC_LIGHT             0           1
...                              ...         ...
196      OFF_LANE                  1           0
198      TRAFFIC_LIGHT             1           0
199      PEDESTRIAN                0           1
         OFF_LANE                  1           0
201      OFF_LANE                  1           0
203      OFF_LANE                  1           0
205      OFF_LANE                  1           0
207      OFF_LANE                  0           1
211      VEHICLES                  0           1
215      OFF_LANE                  1           0
217      VEHICLES                  0           1
218      VEHICLES                  0           1
228      VEHICLES                  0           1
229      VEHICLES                  0           1
230      VEHICLES                  0           1
231      VEHICLES                  0           1
         OFF_LANE                  1           0
246      VEHICLES                  0           1
249      OFF_LANE                  1           0
251      PEDESTRIAN                1           0
255      VEHICLES                  0           1
         OFF_LANE                  1           0
256      PEDESTRIAN                0           1
         VEHICLES                  0           1
         BIKES                     0           1
258      OFF_LANE                  1           0
259      VEHICLES                  0           1
266      VEHICLES                  0           1
269      VEHICLES                  0           1
270      VEHICLES                  0           1

[162 rows x 2 columns]

### Calculate a score for each frame based on the number of false predictions

In [10]:
scores = comparison_table.groupby(['FRAME_ID']).size().reset_index(name='SCORE')
scores =  len(tester_log.columns) - scores['SCORE'] - 1

results['ACCURACY', 'SCORE'] = scores
results['ACCURACY', 'SCORE'].fillna(len(tester_log.columns) - 1, inplace=True) 
results['ACCURACY', 'SCORE'] = results['ACCURACY', 'SCORE'] / 7
results.style.format("{:.2%}", subset=pd.IndexSlice[:, pd.IndexSlice[:, 'SCORE']])

results

PEDESTRIAN                VEHICLES                   BIKES  \
         GROUND TRUTH PREDICTION GROUND TRUTH PREDICTION GROUND TRUTH   
FRAME_ID                                                                
0                   1          1            1          1            0   
1                   0          1            1          1            0   
2                   0          1            1          1            0   
3                   1          1            1          1            0   
4                   0          1            1          1            0   
5                   1          1            1          1            0   
6                   1          1            1          1            0   
7                   1          1            1          1            0   
8                   0          1            1          1            0   
9                   1          1            1          1            0   
10                  1          1            1          1            0   
11                  1          1            1          1            0   
12                  0          1            1          1            0   
13                  1          1            1          1            0   
14                  1          1            1          1            0   
15                  1          1            1          1            0   
16                  0          0            1          1            1   
17                  1          1            1          1            0   
18                  0          0            1          1            0   
19                  0          1            1          1            0   
20                  0          1            1          1            0   
21                  0          1            1          1            0   
22                  0          1            1          1            1   
23                  1          1            0          0            1   
24                  1          1            1          1            1   
25                  0          1            1          1            0   
26                  0          0            1          1            0   
27                  1          1            1          1            0   
28                  1          1            1          1            0   
29                  0          0            1          1            0   
...               ...        ...          ...        ...          ...   
244                 0          0            1          1            0   
245                 0          0            1          1            0   
246                 0          0            0          1            0   
247                 0          0            1          1            0   
248                 0          0            1          1            0   
249                 0          0            1          1            0   
250                 0          0            1          1            0   
251                 1          0            1          1            0   
252                 0          0            1          1            0   
253                 0          0            1          1            0   
254                 0          0            1          1            0   
255                 0          0            0          1            0   
256                 0          1            0          1            0   
257                 0          0            1          1            0   
258                 0          0            1          1            0   
259                 0          0            0          1            0   
260                 0          0            1          1            0   
261                 0          0            1          1            0   
262                 0          0            1          1            0   
263                 0          0            1          1            0   
264                 0          0            1          1            0

## Display some basic statistical analytics

### Overall Performance

In [11]:
pd.DataFrame(results['ACCURACY', 'SCORE'].describe())

,ACCURACY
,SCORE
count,274.000000
mean,0.915537
std,0.114775
min,0.428571
25%,0.857143
50%,1.000000
75%,1.000000
max,1.000000


### Predictions results for Pedstrians

In [12]:
target = results.groupby(results['PEDESTRIAN', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(PEDESTRIAN, PREDICTION)",,,,,,,,
0,230.0,0.937267,0.097490,0.428571,0.857143,1.000000,1.000000,1.0
1,44.0,0.801948,0.131495,0.428571,0.714286,0.857143,0.857143,1.0


### Predictions results for Vehicles

In [13]:
target = results.groupby(results['VEHICLES', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(VEHICLES, PREDICTION)",,,,,,,,
0,4.0,0.821429,0.214286,0.571429,0.678571,0.857143,1.0,1.0
1,270.0,0.916931,0.112798,0.428571,0.857143,1.000000,1.0,1.0


### Predictions results for Bikes

In [14]:
target = results.groupby(results['BIKES', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(BIKES, PREDICTION)",,,,,,,,
0,258.0,0.923034,0.110625,0.428571,0.857143,1.000000,1.000000,1.0
1,16.0,0.794643,0.116277,0.571429,0.714286,0.857143,0.857143,1.0


### Predictions results for Stop Signs

In [15]:
target = results.groupby(results['STOP_SIGN', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(STOP_SIGN, PREDICTION)",,,,,,,,
0,274.0,0.915537,0.114775,0.428571,0.857143,1.0,1.0,1.0


### Predictions results for Traffic Lights

In [16]:
target = results.groupby(results['TRAFFIC_LIGHT', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(TRAFFIC_LIGHT, PREDICTION)",,,,,,,,
0,225.0,0.935238,0.093405,0.571429,0.857143,1.000000,1.0,1.0
1,49.0,0.825073,0.155033,0.428571,0.714286,0.857143,1.0,1.0


### Predictions results for Off-Lane Warnings

In [17]:
target = results.groupby(results['OFF_LANE', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(OFF_LANE, PREDICTION)",,,,,,,,
0,262.0,0.918212,0.113283,0.428571,0.857143,1.000000,1.0,1.0
1,12.0,0.857143,0.136209,0.571429,0.821429,0.857143,1.0,1.0


### Predictions results for Collision Warnings

In [18]:
target = results.groupby(results['COLLISION', 'PREDICTION'], axis=0).describe()
target['ACCURACY', 'SCORE']

,count,mean,std,min,25%,50%,75%,max
"(COLLISION, PREDICTION)",,,,,,,,
0,266.0,0.920516,0.109486,0.428571,0.857143,1.000000,1.000000,1.000000
1,8.0,0.750000,0.166424,0.428571,0.678571,0.857143,0.857143,0.857143
